# Setup

We will be using two repositories:


*   The actual liboqs C-based library: https://github.com/open-quantum-safe/liboqs
*   The Python wrapper for that library: https://github.com/open-quantum-safe/liboqs-python



A wrapper is a piece of code that provides an interface between two different systems or components, in this case liboqs (written in C) and Python.The liboqs-python wrapper is the code written in Python that "wraps around" the C library. Its purpose is to:

*   Translate: It translates calls from Python code into the format that the C library understands.
*   Simplify: It often simplifies the interface of the C library, making it easier to use from Python.
*   Handle Data Types: It manages the conversion of data types between Python and C.
*   Load the Library: It handles loading the shared C library into memory so that Python can access its functions.

In [ ]:
# Install the necessary dependencies
!apt-get update
!apt-get install -y cmake build-essential git

For the wrapper to work correctly, it is required to save the compiled files in the `/root/_oqs` directory, so set the parameter `-DCMAKE_INSTALL_PREFIX="/root/_oqs"`.

In [ ]:
# Configure, build, and install liboqs, per https://github.com/open-quantum-safe/liboqs-python
!git clone https://github.com/open-quantum-safe/liboqs.git # creting liboqs directory in /content
!cmake -S liboqs -B liboqs/build -DCMAKE_INSTALL_PREFIX="/root/_oqs" -DBUILD_SHARED_LIBS=ON # save the compiled files in /root/_oqs
!cmake --build liboqs/build --parallel 8
!sudo cmake --build liboqs/build --target install

In [ ]:
# Install the liboqs python wrapper
!git clone --depth=1 https://github.com/open-quantum-safe/liboqs-python # creting liboqs directory in /content
%cd liboqs-python
!pip install .

In [ ]:
import oqs

In [ ]:
# Get the supported KEM algorithms
oqs.get_enabled_kem_mechanisms()

In [ ]:
# Get the supported signature algorithms
oqs.get_enabled_sig_mechanisms()

# KEMs and Signatures in liboqs

## KEMs

In [ ]:
# Alice creates an instance of the algorithm used as KEM
# This instance contains all the parameters and function pointers for the algorithm to function
alice_kem = oqs.KeyEncapsulation("FrodoKEM-640-AES")

# Generate the KEM keypair
alice_public_key_kem = alice_kem.generate_keypair()
print(f"Alice's KEM public key {alice_public_key_kem.hex()}")
print(f"Size: {len(alice_public_key_kem)} bytes")

print("--------------------")

# Retrieve the KEM private key
private_key_kem = alice_kem.export_secret_key()
print(f"Alice's KEM private key: {private_key_kem.hex()}")
print(f"Size: {len(private_key_kem)} bytes")

# It is possible to reconstruct a session using your private key when creating an instance
kem_later = oqs.KeyEncapsulation("FrodoKEM-640-AES", private_key_kem)

In [ ]:
# Bob encapsulates the symmetric key using Alice's KEM public key
# The output of this function is a randomly generated symmetric key (symmetric_key_bob) and its ciphertext version (ciphertext)
bob_kem = oqs.KeyEncapsulation("FrodoKEM-640-AES")
ciphertext, symmetric_key_bob = bob_kem.encap_secret(alice_public_key_kem)

print(f"Symmetric key: {symmetric_key_bob.hex()}")
print(f"Size: {len(symmetric_key_bob)}")

print("--------------------")

print(f"ciphertext: {ciphertext.hex()}")
print(f"Size: {len(ciphertext)}")

In [ ]:
# Alice decapsulates the ciphertext to obtain the shared secret
symmetric_key_alice = alice_kem.decap_secret(ciphertext)

print(f"Symmetric key: {symmetric_key_alice.hex()}")
print(f"Size: {len(symmetric_key_alice)}")

In [ ]:
# Verify that the secret keys match
if symmetric_key_alice == symmetric_key_bob:
  print("Symmetric keys match! Use it for symmetric encryption.")
else:
  print("Symmetric keys did not match...try again.")

Your are going to creaagree on a shared secret with a colleague to exchange encrypted messages using the quantum resitant algorithm BIKE-L1, which is available in liboqs. To do so, follow the next steps:

- Person A will generate a public key and post it in an entry [here](https://padlet.com/pablogf/hands-on-pqc-at-opensouthcode-4957mbhgd0u8486m).
- Person B will encapsulate a key for Person A using their public key.
  - *Tip: All functions work with bytes. When copying the public key from the dashboard into the colab, make sure to use `bytes.fromhex()`.*
- Person A will decapsulate it, and use it to encrypt a message of your choice using AES-256 CBC mode in [cryptii.com](https://cryptii.com/) (make sure to leave the IV section as it is).
- Person A will post the encrypted message in the comments of that same entry.
- Person B will have to decrypt it using the same tool and configuration. Post the resulting plaintext in the dashbaord to verify it is correct!

Now you have a key to communicate with!

Use the cell below to perform the role of Alice:

In [ ]:
# Create an instance of the algorithm used as KEM

# Generate a KEM keypair


In [ ]:
# Decapsulate


Use the cell below to perform the role of Bob:



In [ ]:
# Encapsulate


## Digital Signatures

In [ ]:
# Alice creates an instance of the algorithm used for signing
alice_sig = oqs.Signature("SPHINCS+-SHA2-128f-simple")

# Alice generates a signature keypair
alice_public_key_sig = alice_sig.generate_keypair()
print(f"Signature public key: {alice_public_key_sig.hex()}")
print(f"Size: {len(alice_public_key_sig)} bytes")

print("--------------------")

# Alice retrives the signature private key
private_key_sig = alice_sig.export_secret_key()
print(f"Signature private key: {private_key_sig.hex()}")
print(f"Size: {len(private_key_sig)} bytes")

In [ ]:
# Sign the message
input_string = "test"
signature = alice_sig.sign(input_string.encode("utf-8"))
print(f"Signature: {signature.hex()}")
print(f"Size: {len(signature)} bytes")

In [ ]:
# Bob creates an instance of the algorithm the message was signed with
bob_sig = oqs.Signature("SPHINCS+-SHA2-128f-simple")

# Bob verifies the signature
is_valid = bob_sig.verify(input_string.encode("utf-8"), signature, alice_public_key_sig)
print(f"Is the signature valid? {is_valid}")

### Your turn!

You are going to create a quantum-resitant signature using the soon-to-be-standardized Falcon-1024 algorithm. Follow the next steps:

- Person A will generate a public key, come up with any message they want, and sign it.
- Person A will then create an entry [here](https://padlet.com/pablogf/hands-on-pqc-at-opensouthcode-4957mbhgd0u8486m) containing the signature, the message being signed, and their public key.
  - Besides this, make sure to post several different tweaked versions of the signature by altering a character of the signature. This way, the verifier will have to find which of the signatures is the correct one.
- Person B will verify the signatures posted, and will respond in a comment to the entry stating which one is the valid signature.
  - *Tip: Remember that all variables must be worked with in bytes. Use `bytes.fromhex()` when pasting the public key and the signature from the dashboard into the notebook. For the input string make sure to use `.decode("utf-8").`*

Use the cell below to perform the role of Alice:

In [ ]:
# Create an instance of the algorithm used for signing

# Generate a signature keypair

# Sign the message


Use the cell below to perform the role of Bob:

In [ ]:
# Verify the signature


# PQC protocol using ML-KEM and ML-DSA

You are now ready to program an authenticated quantum-secure communication protocol using liboqs! Let's do it using the algorithms standardized by NIST: ML-KEM-512 as the key encapsulation mechanism and ML-DSA-44 as digital signature.

## On Alice's end...

Alice generates keys for KEM:

In [ ]:
# Create an instance of the algorithm used as KEM
# This isntance contains all the parameters and function pointers for the algorithm to function
alice_kem = oqs.KeyEncapsulation("ML-KEM-512")

# Alice generates the KEM keypair
alice_kem_PK = alice_kem.generate_keypair()
print(f"Alice's KEM public key {alice_kem_PK.hex()}")
print(f"Size: {len(alice_kem_PK)} bytes")

print("--------------------")

# Retrieve the KEM private key
alice_kem_pk = alice_kem.export_secret_key()
print(f"Alice's KEM private key: {alice_kem_pk.hex()}")
print(f"Size: {len(alice_kem_pk)} bytes")

Alice generates keys for authentication:

In [ ]:
# Create an instance of the algorithm used for signing
alice_sig = oqs.Signature("ML-DSA-44")

# Alice generates a signature keypair
alice_sig_PK = alice_sig.generate_keypair()
print(f"Alice's signature public key: {alice_sig_PK.hex()}")
print(f"Size: {len(alice_sig_PK)} bytes")

print("--------------------")

# Retrive the signature private key
alice_sig_pk = alice_sig.export_secret_key()
print(f"Alice's signature private key: {alice_sig_pk.hex()}")
print(f"Size: {len(alice_sig_pk)} bytes")

Alice signs her public key to authenticate it:

In [ ]:
# Alice signs her KEM public key for authentication and sends it to Bob
signature_alice = alice_sig.sign(alice_kem_PK)
print(f"Alice's signature of her public key: {signature_alice.hex()}")
print(f"Size: {len(signature_alice)}")

## On Bob's end...

Bob verifies Alice's signature:

In [ ]:
# Create an instance of the algorithm that was used for the signature
bob_sig = oqs.Signature("ML-DSA-44")

# Bob verifies Alice's signature
is_alice_real = bob_sig.verify(alice_kem_PK, signature_alice, alice_sig_PK)
print(f"Is Alice who she claims to be? {is_alice_real}")

Then, he moves on to encapsulate a symmetric key:

In [ ]:
# Bob encapsulates the symmetric key using Alice's KEM public key
# The output of this function is: a randomly generated symmetric key (symmetric_key) and its ciphertext version (ciphertext)
bob_kem = oqs.KeyEncapsulation("ML-KEM-512")
ciphertext, symmetric_key_bob = bob_kem.encap_secret(alice_kem_PK)

print(f"Symmetric key: {symmetric_key_bob.hex()}")
print(f"Size: {len(symmetric_key_bob)}")

print("--------------------")

print(f"ciphertext: {ciphertext.hex()}")
print(f"Size: {len(ciphertext)}")

Now he creates a signature of the ciphertext to prove authenticity of the message sent:

In [ ]:
# Create an instance of the algorithm used for signing
#bob_sig = oqs.Signature("ML-DSA-44")

# Bob generates a signature keypair
bob_sig_PK = bob_sig.generate_keypair()
print(f"Bob's signature public key: {bob_sig_PK.hex()}")
print(f"Size: {len(bob_sig_PK)} bytes")

print("--------------------")

# Retrive the signature private key
bob_sig_pk = bob_sig.export_secret_key()
print(f"Bob's signature private key: {bob_sig_pk.hex()}")
print(f"Size: {len(bob_sig_pk)} bytes")

In [ ]:
# Bob signs the ciphertext for authentication and sends it to Alice
signature_bob = bob_sig.sign(ciphertext)
print(f"Bob's signature of the ciphertext: {signature_bob.hex()}")
print(f"Size: {len(signature_bob)}")

## Back to Alice...

Alice verifies the authenticity of Bob's message:

In [ ]:
is_bob_real = alice_sig.verify(ciphertext, signature_bob, bob_sig_PK)
print(f"Is Bob who he claims to be? {is_bob_real}")

Then she decapsulates the symmetric key:

In [ ]:
symmetric_key_alice = alice_kem.decap_secret(ciphertext)

print(f"Symmetric key: {symmetric_key_alice.hex()}")
print(f"Size: {len(symmetric_key_alice)}")

In [ ]:
# Verify that the secret keys match
if symmetric_key_alice == symmetric_key_bob:
  print("Symmetric keys match! Use it for symmetric encryption.")
else:
  print("Symmetric keys did not match...try again.")

Now Alice can use that symmetric key with a symmetric cipher like AES to start to securely communicate with Bob:

In [ ]:
# Install pycryptodome for symmetric encryption with AES
!pip install pycryptodome

In [ ]:
# Retrieve message from the user

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes

message = input("Type a message: ")

# Generate a random initialization vector (IV)
iv = get_random_bytes(16)

# Create the cipher object
cipher = AES.new(symmetric_key_alice, AES.MODE_CBC, iv)

# Pad the message to AES block size (16 bytes) and encrypt
padded_data = pad(message.encode('utf-8'), AES.block_size)
ciphertext = cipher.encrypt(padded_data)

# Combine IV and ciphertext (IV is needed for decryption)
encrypted_data = iv + ciphertext

print("Message to encrypt: ", encrypted_data.hex())

## Back to Bob...

Bob decrypts the cipehertext from Alice with the shared secret acquired during the key encapsualtion process:

In [ ]:
from Crypto.Util.Padding import unpad

ciphertext_hex = input("Ciphertext to decrypt: ")

# Change ciphertext from base64 to bytes
encrypted_data = bytes.fromhex(ciphertext_hex)

# Split IV and ciphertext
iv = encrypted_data[:16]
ciphertext = encrypted_data[16:]

# Decrypt
cipher = AES.new(symmetric_key_alice, AES.MODE_CBC, iv)
decrypted_padded = cipher.decrypt(ciphertext)

# Remove padding
plaintext = unpad(decrypted_padded, AES.block_size)

print("Decrypted message:", plaintext.decode('utf-8'))